In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker
from cycler import cycle

%matplotlib widget

This example uses matplotlib 2.0

In [2]:
%matplotlib widget
import sys
print("Python version is {} \n".format(sys.version))
print("Matplotlib version is {} \n".format(mpl.__version__))
print("Matplotlibrc file is {}\n".format(mpl.matplotlib_fname()))

Python version is 3.9.2 | packaged by conda-forge | (default, Feb 21 2021, 05:02:46) 
[GCC 9.3.0] 

Matplotlib version is 3.4.2 

Matplotlibrc file is matplotlibrc



and the matplotlibrc file from 
https://github.com/mpmdean/matplotlibrc
please suggest improvements

## A few notes

* All fields are using latex by default -- this avoid non-uniformity when using latex in some labels. 

* Using a uniform style is very important for an attractive manuscript. Try to avoid ad-hoc changes to the settings. 

* If you feel that there is a desirable change, consider applying it to the whole manuscript, preferably by setting some edits to the rc file right at the start of the script/notebook. e.g. mpl.rc('font', size=10). 

## Make fake data

In [3]:
Ts = np.linspace(20, 100, 5)
x = np.linspace(0, 10, 10)
ys = [np.exp(-(x-2)/3) + (np.random.rand(x.size)-0.5)*0.3 for T in Ts]
fits = [np.exp(-(x-2)/3) for T in Ts]

X, Y = np.meshgrid(x,x)
M = np.exp(-np.sqrt((X-2)**2 + (Y-3)**2))
M += np.random.rand(*M.shape)*0.1

## Create a set of colors and a cyler of markers
It's preferable to use color in a meaningful and consistent way throughout a manuscript. For example all data at 20 K is black etc. 

In [4]:
colors = mpl.cm.inferno(np.linspace(0, 0.8, len(ys))) # avoid yellow due to visibility
markers = cycle(['o', 's', '^', 'v', 'p', '<', '>', 'h'])

## Create figure

* The width of a figure should always be set to the real physical column width. e.g. 3+3/8 inches for a PRL column. This will mean that fonts appear on the page in the correct size.

* Gridspec and subplots_adjust are useful to set the precise location of panels.

* Including 2-4 minor ticks per major tick is usually good.

* Putting the colorbar on separate axes facilities better control over its location. 

* Saving in pdf creates vector graphics and fast performance within pdflatex.

* It sometimes makes sense to flip the order of legend entries to match the order if which they appear on the plot.

In [5]:
fig = plt.figure(figsize=(3+3/8, 4))

gs = GridSpec(3,2, width_ratios=[1.2,0.8], height_ratios=[1,0.1,1])

ax0 = plt.subplot(gs[:,0])

offsets = np.arange(len(Ts)) * 1
for y, fit, T, offset, color, marker in zip(ys, fits, Ts, offsets, colors, markers):
    ax0.plot(x, fit + offset, '-', color=color, alpha=0.5)
    ax0.plot(x, y + offset, color=color, marker=marker, linestyle='None', label="T = {:.0f} K".format(T))
    
ax0.set_xlabel(r'$\tau$ (s)')
ax0.set_ylabel(r'$g_2(\tau)$')

ax0.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax0.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax0.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))


ax1 = plt.subplot(gs[0,1])
art = ax1.pcolormesh(X, Y, M, vmin=0, vmax=0.5, shading='auto')
ax1.set_axis_off()
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))

ax2 = plt.subplot(gs[1,1])
cb = plt.colorbar(art, cax=ax2, orientation="horizontal")
cb.set_label('I  (ADU)')
cb.set_ticks(ticker.MultipleLocator(0.2))


ax3 = plt.subplot(gs[2,1])
handles, labels = ax0.get_legend_handles_labels()
ax3.axis('off')
ax3.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.1,0.8))

fig.text(0.1, 0.9, '(a)')
fig.text(0.65, 0.9, '(b)')

fig.subplots_adjust(wspace=0.1, hspace=0.1, left=0.1, right=0.97)

fig.savefig('ex_01_multipanel_colorplot.pdf', dpi=1200)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …